In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interp

import time_to_H

In [ ]:
# Read data

folder_fields = "/media/anais/Data/Alexandre/data/CCGM_500um_375nm_45uN_100Pa_h_1cm_m_93g/muDIC"

dt_camera = 0.1

frames_true = np.load(os.path.join(folder_fields, "frames.npy"))
coords = np.load(os.path.join(folder_fields, "coords.npy"))
strain = np.load(os.path.join(folder_fields, "true_strain.npy"))

times_true = dt_camera * frames_true
height_calib = time_to_H.time_to_H(times_true)

with open(os.path.join(folder_fields, 'params.txt')) as file:
    params = file.read()

In [ ]:
# Data shape

print(f"Frames computed: {frames_true.tolist()}")
print(f"Coords of shape {coords.shape} -> (useless, x or y, x_elem, y_elem, frame)")
print(f"Strain of shape {strain.shape} -> (useless, ?, ?, x_elem, y_elem, frame)")

In [ ]:
# Plot the strain

frame_index = -1
frame_true = frames_true[frame_index]

x0 = coords[0,0,:,:,0]
y0 = coords[0,1,:,:,0]

strain_toPlot = strain[0,0,0,:,:,frame_index]

fig, ax = plt.subplots()
ax.set_title(f"Strain [frame {frame_true}]")

im = ax.contourf(x0, y0, strain_toPlot, levels=100)
fig.colorbar(im, ax=ax, label="Stain")

ax.set_xlabel('Position x [pix]')
ax.set_ylabel('Position y [pix]')

plt.show()

In [ ]:
# Plot the strain along given lines

y_pix_list = [800, 1000, 1200, 1400]

# Find nearest element for each y_pix
y_elem_list = []
for y_pix in y_pix_list:
    y_elem = np.argmin(np.abs(y_pix - y[y.shape[0]//2,:]))
    y_elem_list.append(y_elem)

# Plot
fig, ax = plt.subplots(tight_layout=True)

for y_pix, y_elem in zip(y_pix_list, y_elem_list):
    ax.plot(x0[:,y_elem], strain_toPlot[:,y_elem], label=f'y_pix = {y_pix}')

ax.legend()

plt.show()

## True strain, with removed calibration

In [ ]:
# Read calibration

pathCalibration = "/media/anais/Data/Alexandre/data/Calibrations/Calibration_top_grey/muDIC"

dt_camera_calib = 0.1

frames_true_calib = np.load(os.path.join(pathCalibration, "frames.npy"))
coords_calib = np.load(os.path.join(pathCalibration, "coords.npy"))
strain_calib = np.load(os.path.join(pathCalibration, "true_strain.npy"))

times_true_calib = dt_camera_calib * frames_true_calib
height_calib = time_to_H.time_to_H(times_true_calib)


In [ ]:
# Print shapes

print(f"Frames: {frames_true_calib.shape}")
print(f"Coords: {coords_calib.shape}")
print(f"Strain: {strain_calib.shape}")

In [ ]:
# Average along x

strain_calib_xx = strain_calib[0,0,0,:,:,:].mean(axis=1)
x0_calib = coords_calib[0,0,:,:,0].mean(axis=1)

print(strain_calib_xx.shape, x0_calib.shape)

plot = True

if plot:

    fig, ax = plt.subplots()

    ax.plot(x0_calib, strain_calib_xx[:,::10])

    plt.show()

In [ ]:
# Create interpolator (x0 [pix], time) ->  strain_calib

# Strain
strain_calib_interp = interp.RegularGridInterpolator((x0_calib, times_true_calib), strain_calib_xx, 
                                              bounds_error=False, fill_value=None)

strainPrev_calib_xx = np.gradient(strain_calib_xx, axis=1)
strainPrev_calib_interp = interp.RegularGridInterpolator((x0_calib, times_true_calib), strainPrev_calib_xx, 
                                              bounds_error=False, fill_value=None)

strainRate_calib_xx = np.gradient(strain_calib_xx, times_true_calib, axis=1)
strainRate_calib_interp = interp.RegularGridInterpolator((x0_calib, times_true_calib), strainRate_calib_xx, 
                                              bounds_error=False, fill_value=None)

In [ ]:
# Test interpolation

x0_plot_ = np.linspace(400, 1900, 100)
times_true_plot_ = np.linspace(0, 50, 10)

x0_plot, times_true_plot = np.meshgrid(x0_plot_, times_true_plot_, indexing='ij')

strain_from_interp = strain_calib_interp((x0_plot.flatten(), times_true_plot.flatten())).reshape(x0_plot.shape)
strainRate_from_interp = strainRate_calib_interp((x0_plot.flatten(), times_true_plot.flatten())).reshape(x0_plot.shape)

fig, ax = plt.subplots(figsize=(11,5), ncols=2)

ax[0].plot(x0_plot_, strain_from_interp)
ax[1].plot(x0_plot_, strainRate_from_interp)

ax[0].set_title("Strain global")
ax[1].set_title("Strain rate")

plt.show()


In [ ]:
# Plot strain with calibration removed

frame_index = -1
frame_true = frames_true[frame_index]
time_true = times_true[frame_index]

print(time_true)

x0 = coords[0,0,:,:,0]
y0 = coords[0,1,:,:,0]

strain_toPlot = strain[0,0,0,:,:,frame_index]

# Remove calibration from strain_toPlot
x0_flat = x0.flatten()
strain_visual = strain_calib_interp((x0_flat, np.full_like(x0_flat, time_true))).reshape(x0.shape)

fig, ax = plt.subplots(figsize=(15,5), ncols=2)

ax[0].set_title(f"Strain raw [frame {frame_true}]")
ax[1].set_title(f"Strain corrected [frame {frame_true}]")

im = ax[0].contourf(x0, y0, strain_toPlot)
fig.colorbar(im, ax=ax[0], label="Stain")

im = ax[1].contourf(x0, y0, strain_toPlot - strain_visual)
fig.colorbar(im, ax=ax[1], label="Stain")

for axi in ax:
    axi.set_xlabel('Position x [pix]')
    axi.set_ylabel('Position y [pix]')

plt.show()

In [ ]:
# Plot the strain along given lines

y_pix_list = [800, 1000, 1200, 1400]

# Find nearest element for each y_pix
y_elem_list = []
for y_pix in y_pix_list:
    y_elem = np.argmin(np.abs(y_pix - y0[y0.shape[0]//2,:]))
    y_elem_list.append(y_elem)

# Plot
fig, ax = plt.subplots(tight_layout=True)

for y_pix, y_elem in zip(y_pix_list, y_elem_list):
    # Remove calibration from strain_toPlot
    strain_visual = strain_calib_interp((x0[:,y_elem], np.full_like(x0[:,y_elem], time_true)))

    ax.plot(x0[:,y_elem], strain_toPlot[:,y_elem] - strain_visual, label=f'y_pix = {y_pix}')

ax.legend()

plt.show()